# Workflow setup

In this notebook we will set up the workflows for all of the bulk metals and corresponding slab structures from [De Waele et al](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.94.235418).

The functions defined in the `support_functions.py` module are used to either load the slab structures from the `.cif` files of [De Waele et al](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.94.235418), or generate the slabs in the `structure_dict` from the bulk structure file specified.

In [1]:
# Notebook header  <---- Always execute this cell first!

import os, re
import numpy as np
import ruamel.yaml as yaml

from warnings import warn

from pymatgen import Structure
from custodian.vasp.handlers import VaspErrorHandler, NonConvergingErrorHandler, UnconvergedErrorHandler

from vscworkflows.config import load_config
from vscworkflows.workflows.core import get_wf_quotas
from vscworkflows.handlers.core import ElectronicConvergenceMonitor, QuotasErrorHandler

from support_functions import get_dewaele_slab_list, get_generated_slab_list


## Loading the bulk/slab structures

In order to construct the workflow, we need the bulk structure and slab structures, along with the number of fixed layers in the center of each slab. The cell below loads a dictionary that specifies which bulk structure files to use, as well as the number of fixed layers for each element and miller index, based on the convergence information in the supplementary information of [De Waele et al](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.94.235418).

In [2]:
with open("../data/structure_dict.yaml", "r") as file:
    structure_dict = yaml.safe_load(file.read())

## Submit Workflow

Next we need to submit the workflow to the preferred mongoDB server that hosts our calculations.

In [3]:
# Set the calculation settings

lpad = load_config("launchpad", "test")
scratch_dir = "/scratch/antwerpen/202/vsc20248/quotas"

functional=('pbe', {})

handlers = [VaspErrorHandler(errors_subset_to_catch=["zbrent"]), 
            UnconvergedErrorHandler(), 
            NonConvergingErrorHandler(nionic_steps=3), 
            ElectronicConvergenceMonitor(max_fit_range=100),
            QuotasErrorHandler()]

In [5]:
# Choose the element for the workflow

element = "Al"
magnetic_moment = None # Only needs to be specified in case the slabs 
                       # are loaded from dewaele's cif files
slab_choice = "self_generated" # Either "dewaele" or "self_generated"

bulk = Structure.from_file(
    os.path.join("../data/cifs", element, structure_dict[element]["bulk"])
)

if slab_choice == "self_generated":
    slab_list = get_generated_slab_list(
        element, structure_dict, "../data/cifs/"
    )
elif slab_choice == "dewaele":
    slab_list = get_dewaele_slab_list(
        element, structure_dict, "../data/cifs/"
    )
else:
    raise ValueError()
        
if slab_choice == "dewaele" and magnetic_moment is not None:
    for slab_dict in slab_list:
        slab_dict["slab"].add_site_property("magmom", [2] * len(slab_dict["slab"]))

In [7]:
lpad.add_wf(
    get_wf_quotas(
        bulk=bulk,
        slab_list=slab_list,
        directory=os.path.join(scratch_dir, slab_choice, element),
        functional=functional,
        is_metal=True,
        in_custodian=handlers
    ))

2020-01-01 10:54:51,151 INFO Added a workflow. id_map: {-16: 1236, -15: 1237, -14: 1238, -13: 1239, -12: 1240, -11: 1241, -10: 1242, -9: 1243}


{-16: 1236,
 -15: 1237,
 -14: 1238,
 -13: 1239,
 -12: 1240,
 -11: 1241,
 -10: 1242,
 -9: 1243}

### Separate surfaces - DOS workflow

For some of the problematic surfaces, we want to set up a separate DOS workflow where we specify the user_incar_settings to improve the convergence. Note that the `ElectronicConvergenceMonitor` is not used here, as some of these surfaces simply require a very large amount of electronic steps to converge, and might show signs of not converging even after a couple of hundred electronic steps.

In [14]:
from vscworkflows.workflows.core import get_wf_slab_dos

lpad = load_config("launchpad", "test")
scratch_dir = "/scratch/leuven/202/vsc20248/"

handlers = [VaspErrorHandler(errors_subset_to_catch=["zbrent"]), 
            UnconvergedErrorHandler(), 
            NonConvergingErrorHandler(nionic_steps=3), 
            #ElectronicConvergenceMonitor(max_interp_range=100),
            QuotasErrorHandler()]

user_incar_settings = {
    "ALGO": "All",
    "NELM": 1000,
    "NELMDL": -8,
}

In [18]:
element = "Cr_afm"
miller_index = (1, 1, 0)
slab_choice = "self_generated"
magnetic_moment = None

if slab_choice == "self_generated":
    slab_list = get_generated_slab_list(
        element, structure_dict, "../data/cifs/"
    )
elif slab_choice == "dewaele":
    slab_list = get_dewaele_slab_list(
        element, structure_dict, "../data/cifs/"
    )
else:
    raise ValueError()

slab_dict = None

for slab in slab_list:
    try:
        if slab["slab"].miller_index == miller_index:
            slab_dict = slab
    except AttributeError:
        print("Found str instead of slab. This indicates multiple terminations. " +
              "Figure out how to deal with this!")
        
if slab_choice == "dewaele" and magnetic_moment is not None:
    slab_dict["slab"].add_site_property("magmom", [2] * len(slab_dict["slab"]))

In [19]:
wf = get_wf_slab_dos(
    slab=slab_dict["slab"], 
    directory=os.path.join(scratch_dir, "quotas", slab_choice, element, "".join([str(i) for i in miller_index])),
    user_slab_settings=slab_dict["user_slab_settings"],
    calculate_locpot=True,
    is_metal=True,
    user_incar_settings=user_incar_settings,
    in_custodian=handlers,
    auto_parallelization=True
)

In [20]:
lpad.add_wf(wf)

2019-10-29 12:55:00,977 INFO Added a workflow. id_map: {-8: 23, -7: 24}


{-8: 23, -7: 24}